In [3]:
import pdfplumber
import jieba
import jieba.posseg as pseg #词性标注
import jieba.analyse as anls #关键词提取
from sklearn.cluster import KMeans
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from docx import Document

import re
import os
from collections import Counter

def pdf_parse_table(path):
    """解析pdf文件中的表格"""
    origin = pdfplumber.open(path)
    tables = []
    for page in origin.pages:
        # 获取当前页面的全部表格
        table_page = [table for tables in page.extract_tables() for table in tables if len(table)!=0]
        tables.append(table_page)
    origin.close()
    return tables

def pdf_parse_text(path):
    """解析pdf为"""
    origin = pdfplumber.open(path)
    texts = []
    for page in origin.pages:
        text_page = [text.replace(' ','') for text in page.extract_text().split('\n') if len(text.replace(' ','')) != 0]
        texts.extend(text_page)
    origin.close()
    return texts

def re_match(texts):
    """匹配技术部分小节"""
    pattern = "第.*节"
    regex = re.compile(pattern)
    sections = []
    index = 0
    for text in texts:
        if len(regex.findall(text)) > 0:
            sections.append((index,text))
#             print(text,index)
        index+=1
    return sections

def get_sections(dirname):
    """获取小节"""
    sections = {}
    text_extract = {}
    for _,_,files in os.walk(dirname):
        for filename in files:
            if filename.split('.')[-1] == 'pdf':
                texts = pdf_parse_text(dirname+'/'+filename)
                text_extract[filename.split('.')[0]] = texts
                sections_ = re_match(texts)
                sections[filename.split('.')[0]] = sections_
                print("{}小节获取完毕".format(filename))
    return sections,text_extract

def get_exact_section(dirname,numberlist):
    """获取解析的小节列表"""
    sections = []
    for _,_,files in os.walk(dirname):
        for filename in files:
            if filename.split('.')[-1] == 'pdf':
                texts = pdf_parse_text(dirname+'/'+filename)
                try:
                    section_exact = texts[numberlist[filename.split('.')[0]][0]:numberlist[filename.split('.')[0]][1]]
                    sections.append((filename.split('.')[0],section_exact))
                except:
                    pass
                print("{}小节获取完毕".format(filename))
    return sections
def get_cross(sections):
    """获取公有的篇章"""
    set1 = {sectionname[3:] for _, sectionname in list(sections.values())[0]}
    for label, sections_ in sections.items():
        set2 = {sectionname[3:] for _, sectionname in list(sections_)}
        if (len(set2) > 0):
            # print(set2)
            set1 = set1 & set2
            print(set1)
    counter = Counter()
    for label, sections_ in sections.items():
        set2 = {sectionname[3:] for _, sectionname in list(sections_)}
        if (len(set2) > 0):
            set3 = set2 - set1
            counter.update(set3)
    summary = len(sections)
    for item, count in counter.items():
        if (count > summary * 0.5):
            set1.update({item})
    return set1
def get_section_matiral_index(sections,extracts:set):
    """
    获取章节的索引
    :param sections:
    :param extracts:
    :return:
    """
    result = {}
    for extract in extracts:
        section_matriels = []
        for label,sections_ in sections.items():
            regex = re.compile(extract)
            for index in range(len(sections_)):
                if len(regex.findall(sections_[index][1])):
                    start = sections_[index][0]
                    if index != len(sections_) - 1:
                        end = sections_[index + 1][0]
                    else:
                        end = start
                    if end - start > 1: # 过滤目录
                        section_matriels.append((label,(start,end)))
        result[extract] = section_matriels
    return result
def get_matiral(matriel_index,texts,extracts):
    """
    获得章节素材，进一步处理
    :param matriel_index:
    :param texts:
    :param extracts:
    :return:
    """
    matirals = {}
    for extract in extracts:
        temp = []
        for label,(start,end) in matriel_index[extract]:
            print(texts[label][start:end])
            temp.append(texts[label][start:end])
        matirals[extract] = temp
    return matirals

def analyse_tfidf(matierals,extracts):
    """tfidf算法下的关键词提取"""
    for extract in extracts:
        matirel = matierals[extract]
        text = ""
        text_item = ""
        for texts in matirel:
            text_item = "".join(texts)
        text += text_item
        text += " "
        anls.set_stop_words('../source/stopwords.txt')
        tags = anls.extract_tags(text, topK=100, withWeight=True)
        with open("../source/tfidf.txt", "w", encoding="utf-8") as f:
            f.write("分析的章节：{}\n".format(extract))
            f.write("\n".join(["{},rate:{}".format(tag[0],tag[1]) for tag in tags]))
def analyse_textRank(matierals,extracts):
    """textRank算法下的关键词提取"""
    for extract in extracts:
        matirel = matierals[extract]
        text = ""
        text_item = ""
        for texts in matirel:
            text_item = "".join(texts)
        text += text_item
        text += " "
        anls.set_stop_words('../source/stopwords.txt')
        tags = anls.textrank(text, topK=100, withWeight=True)
        with open("../source/textrank.txt","w",encoding="utf-8") as f:
            f.write("分析的章节：{}\n".format(extract))
            f.write("\n".join(["{},rate:{}".format(tag[0],tag[1]) for tag in tags]))
def stop_words(path):
    """停用词"""
    stopWords = []
    with open(path,encoding='utf-8') as f:
        stopWords = [line.strip()for line in f.readlines()]
    return stopWords
def analyse_pseg(matierals,extracts):
    """词性标注分析"""
    for extract in extracts:
        matirel = matierals[extract]
        text = ""
        text_item = ""
        stopWords = stop_words('../source/stopwords.txt')
        for texts in matirel:
            text_item = "".join(texts)
        text += text_item
        words = pseg.cut(text,use_paddle=True)
        for word, flag in words:
            if word.strip() not in stopWords and len(word.strip()) > 1:
                print('%s %s' % (word, flag))

def cut_word(texts,stopWords):
    """
    对句子进行分词
    :param texts:
    :param stopWords:
    :return:
    """
    cut_list = []
    for text in texts:
        for word in list(jieba.cut(text)):
            if word.strip() not in stopWords and len(word.strip()) > 1:
                cut_list.append(word)
    return cut_list
def cluster(x_train,modelPath):
    """聚类筛选大样本"""
    doc_vectors = []
    model = Doc2Vec.load(modelPath)
    matirel_dict = []
    print("success loading Doc2Vec model")
    for text,label in x_train:
        vec = model.infer_vector(text)
        doc_vectors.append(vec)
        matirel_dict.append(text)
    # print(doc_vectors)
    clf = KMeans(n_clusters=2)
    clf.fit(doc_vectors)
    print("KMeans 聚类结束")
    return clf,matirel_dict
def means(matierals,extracts):
    result_template = {}
    for extract in extracts:
        matirel = matierals[extract]
        x_train = []
        stopWords = stop_words('../source/stopwords.txt')
        index = 0
        for texts in matirel:
            text_item = ""
            text_item = "".join(texts)
            cut_list = [word for word in list(jieba.cut(text_item)) if word.strip() not in stopWords and len(word.strip()) > 1]
            x_train.append(TaggedDocument(cut_list,tags=[index]))
            index += 1
        model = Doc2Vec(x_train, min_count=1, window=6, vector_size=70, sample=1e-3, negative=5, workers=4)
        model.train(x_train, total_examples=model.corpus_count, epochs=70)
        model.save("../source/models/model_{}".format(extract))
        estimator,matirel_dict = cluster(x_train,"../source/models/model_{}".format(extract))
        counter = Counter(estimator.labels_)
        #print(matirel_dict)
        max_value = 0 if max(counter.values()) == counter[0] else 1
        max_len = 0
        for index in range(len(estimator.labels_)):
            if max_value == estimator.labels_[index] and len(matirel[index])>max_len:
                result_template[extract] = "\n".join(matirel[index])
                max_len = len(matirel[index])
    return result_template

In [4]:
sections,text_extract = get_sections("../source/datas")

9.pdf小节获取完毕
8.pdf小节获取完毕
16.pdf小节获取完毕
15.pdf小节获取完毕
14.pdf小节获取完毕
10.pdf小节获取完毕
11.pdf小节获取完毕
13.pdf小节获取完毕
12.pdf小节获取完毕
6.pdf小节获取完毕
7.pdf小节获取完毕
5.pdf小节获取完毕
4.pdf小节获取完毕
1.pdf小节获取完毕
3.pdf小节获取完毕
2.pdf小节获取完毕


In [5]:
sections,text_extract

({'9': [(1, '第一节供货范围、技术规格、参数与要求'),
   (2, '第二节备件和工具'),
   (3, '第三节设计联络会及配套责任'),
   (4, '第四节设备出厂前检验'),
   (5, '第五节技术服务'),
   (6, '第六节安装、检验、调试、试运行及验收'),
   (7, '第七节质量保证'),
   (8, '第八节技术资料和图纸'),
   (9, '第九节标准'),
   (11, '第一节供货范围、技术规格、参数与要求'),
   (78, '第一节采煤机技术参数及要求'),
   (1332, '第一节采煤机技术参数及要求'),
   (2173, '第一节采煤机技术参数及要求'),
   (2867, '第二节备件和工具'),
   (2889, '第三节设计联络会及配套责任'),
   (2906, '第四节设备出厂前检验'),
   (2920, '第五节技术服务'),
   (2945, '第六节安装、检验、调试、试运行及验收'),
   (2960, '第七节质量保证'),
   (2967, '第八节技术资料和图纸'),
   (2995, '第九节标准')],
  '8': [],
  '16': [(2, '第1节货物需求一览表及工作环境'),
   (77, '第二节采煤机技术参数及要求'),
   (303, '第三节备件和工具'),
   (322, '第四节设备出厂前检验'),
   (335, '第五节技术服务'),
   (360, '第六节安装、检验、调试、试运行及验收'),
   (373, '第七节质量保证'),
   (380, '第八节技术资料和图纸'),
   (408, '第九节标准')],
  '15': [(3, '第一节供货范围、技术规格、参数与要求'),
   (4, '第二节备件和工具'),
   (5, '第三节设计联络会及配套责任'),
   (6, '第四节设备出厂前检验'),
   (7, '第五节技术服务'),
   (8, '第六节安装、检验、调试、试运行及验收'),
   (9, '第七节质量保证'),
   (10, '第八节技术资料和图纸'),
   (11, '第九节标准'),
   (12, '第十节运营及消耗指标'),
   (14, '第

In [6]:
set_section = get_cross(sections)
set_section

{'技术服务', '设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '设计联络会及配套责任', '采煤机技术参数及要求', '供货范围、技术规格、参数与要求', '质量保证'}
{'设备出厂前检验', '标准', '采煤机技术参数及要求', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务', '质量保证'}
{'设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '技术服务'

{'供货范围、技术规格、参数与要求',
 '备件和工具',
 '安装、检验、调试、试运行及验收',
 '技术服务',
 '技术资料和图纸',
 '标准',
 '设备出厂前检验',
 '设计联络会及配套责任',
 '质量保证'}

In [7]:
matriel_index = get_section_matiral_index(sections,set_section)
matriel_index
for label,(start,end) in matriel_index['供货范围、技术规格、参数与要求']:
    print(label)
    print(text_extract[label][start:end])

9
['第一节供货范围、技术规格、参数与要求', '第一包第1-1项：采煤机（截割功率650kW）', '第一部分货物需求一览表', '单数交货', '序号名称规格型号备注', '位量期', '2018', '采煤机1台年8月梅花井煤矿', '30日', '650*2+125*2+20*2', '≥MG650/', '1采煤机+160（进口电机≥', '1750-WD', '620/1660）', '左右摇臂、左右牵引装置配', '套电气设备、各个驱动装置齿轨轮节距', '1.1套1', '的驱动电机、液压系统、截176mm', '割滚筒（包括截齿）等', '顺槽数据传输、接收及显示含缆线及连接件和', '1.2套1', '装置数据传输软件', '自动截割、惯性导航控制及', '1..3套1', '软件', '1.4摄像系统套1', '保证150mm2进', '线，200米，选用', '1.5电缆进线拖曳装置O2C套1霸州华宇或神华泉', '州公司或同等质量', '产品', '选用虎牌电缆或同', '等品质进口电缆，', '100', '1.6电缆150mm2米芯线含高强度通信', '0', '光纤、带电缆连接', '器', 'DN50/38MP', '1.7水管米400', 'a', '2随机大部件', '摇臂(左右摇臂各一件，含', '2.1对1备用', '连接件)', '牵引部及行走箱（左右各一', '2.2对1备用', '套）', '2.3变频器套1备用', '2.4调高油缸件2备用', '44', '2.5调高泵台1备用', '2.6截割滚筒（左右含截齿）Φ1800mm对1备用', '2.7牵引电动机台1备用', '2.8截割电动机台1备用', '2.9油泵电动机台1备用', '3备品备件（序号1总价5%）批1', '4专用工具套2高压泵一台', '5随机技术资料套66套', '第二部分工作环境', '（一）工作面煤层情况', '梅花井煤矿1110208工作面回采走向长4500m，倾斜长约为217m；煤层厚度为3～4m，', '平均3.5m，工作面平均倾角约为20°。', '（二）煤层、岩性情况', '煤层直接顶板为粉砂岩，浅白色，巨厚层状，成分以石英、长石为主，参差状断口，钙', '质胶结，半坚硬，厚度0-12m，平均4.65m。', '第三部分设备主要

In [8]:
def to_json(filepath):
    bids_jsons = {}
    bids_jsons['content'] = ""
    origin = pdfplumber.open(filepath)
    for page in origin.pages:
        text_page = [text.replace(' ','') for text in page.extract_text().split('\n') if len(text.replace(' ','')) != 0]
        bids_jsons['content'] += "\n".join(text_page)
    origin.close()
    return bids_jsons
json_x = to_json("../source/datas/1.pdf")

In [24]:
# 构建词袋模型
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()
# cv_fit = cv.fit_transform([json_x['content']])
# print(sorted(cv.vocabulary_.items(),key=lambda x:x[1]))
# print(cv_fit.toarray())

In [42]:
len(json_x['content'])

10782

In [10]:
result = get_matiral(matriel_index,text_extract,set_section)
result

['第四节设备出厂前检验', '1.为了对合同设备及其相关设备生产期间的质量检验，招标人有权派人到中标人所在工厂', '进行检验。对于在中标人所在地的交通费用和为便于招标人质检要求，诸如必要的安全', '用具、办公用品、技术文件和图纸、核算数据、制造和检验标准及其它必备的检验数据', '应由中标人免费提供。', '2.在制造期间招标人的一切监理和质检活动所形成的书面资料均不作为中标人产品质量证', '明文件。在交货前招标人的质检，既不能免去合同中属于投标人质量担保期范围内的责', '任，也不能替代设备抵运招标人现场的质量检验。', '3．在中检中质检团成员发现或提出的问题，双方应积极通过友好的态度协商解决。', '4.设备在出厂前必须进行整体联合试运转，根据试运转时间确定招标人中检时间，联合试', '运转应在招标人中检人员监督下进行。', '5.在设备到达招标人现场后组装试运转中如出现问题，原因是中标人没有在出厂前进行设', '备整体联合试运转，因此推迟的时间将按照推迟交货期来计算。', '127']
['第四节设备出厂前检验', '1.为了对合同设备及其相关设备生产期间的质量检验，招标人有权派人到中标人所在工厂进', '行检验。为便于招标人质检要求，诸如必要的安全用具、办公用品、技术文件和图纸、核算数', '据、制造和检验标准及其它必备的检验数据应由中标人提供。', '2.在制造期间招标人的一切监理和质检活动所形成的书面资料均不作为中标人产品质量证明', '文件。在交货前招标人的质检，既不能免去合同中属于投标人质量担保期范围内的责任，也不', '能替代设备抵运招标人现场的质量检验。', '3．在中检中质检团成员发现或提出的问题，双方应积极通过友好的态度协商解决。', '4.设备在出厂前必须进行整体联合试运转，根据试运转时间确定招标人终检时间，联合试运', '转应在招标人终检人员监督下进行。', '5.在设备到达招标人现场后组装试运转中如出现问题，原因是中标人没有在出厂前进行设备', '整体联合试运转，因此推迟的时间将按照推迟交货期来计算。', '37']
['第四节设备出厂前检验', '1.在合同设备及其相关设备的生产制造期间如需到国外进行质量检查，招标人可派出一行7', '个成员包括1名翻译到中标人国逗留14天，招标人质量检查人员的费用包括返程机票费、', '膳

{'设备出厂前检验': [['第四节设备出厂前检验',
   '1.为了对合同设备及其相关设备生产期间的质量检验，招标人有权派人到中标人所在工厂',
   '进行检验。对于在中标人所在地的交通费用和为便于招标人质检要求，诸如必要的安全',
   '用具、办公用品、技术文件和图纸、核算数据、制造和检验标准及其它必备的检验数据',
   '应由中标人免费提供。',
   '2.在制造期间招标人的一切监理和质检活动所形成的书面资料均不作为中标人产品质量证',
   '明文件。在交货前招标人的质检，既不能免去合同中属于投标人质量担保期范围内的责',
   '任，也不能替代设备抵运招标人现场的质量检验。',
   '3．在中检中质检团成员发现或提出的问题，双方应积极通过友好的态度协商解决。',
   '4.设备在出厂前必须进行整体联合试运转，根据试运转时间确定招标人中检时间，联合试',
   '运转应在招标人中检人员监督下进行。',
   '5.在设备到达招标人现场后组装试运转中如出现问题，原因是中标人没有在出厂前进行设',
   '备整体联合试运转，因此推迟的时间将按照推迟交货期来计算。',
   '127'],
  ['第四节设备出厂前检验',
   '1.为了对合同设备及其相关设备生产期间的质量检验，招标人有权派人到中标人所在工厂进',
   '行检验。为便于招标人质检要求，诸如必要的安全用具、办公用品、技术文件和图纸、核算数',
   '据、制造和检验标准及其它必备的检验数据应由中标人提供。',
   '2.在制造期间招标人的一切监理和质检活动所形成的书面资料均不作为中标人产品质量证明',
   '文件。在交货前招标人的质检，既不能免去合同中属于投标人质量担保期范围内的责任，也不',
   '能替代设备抵运招标人现场的质量检验。',
   '3．在中检中质检团成员发现或提出的问题，双方应积极通过友好的态度协商解决。',
   '4.设备在出厂前必须进行整体联合试运转，根据试运转时间确定招标人终检时间，联合试运',
   '转应在招标人终检人员监督下进行。',
   '5.在设备到达招标人现场后组装试运转中如出现问题，原因是中标人没有在出厂前进行设备',
   '整体联合试运转，因此推迟的时间将按照推迟交货期来计算。',
   '37'],
  ['第四节设备出厂前检验

In [43]:
template = means(result,set_section)
template

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/lx/t132nn412vn2w7nmcz9g3g3w0000gn/T/jieba.cache
Loading model cost 0.984 seconds.
Prefix dict has been built successfully.


success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束
success loading Doc2Vec model
KMeans 聚类结束


{'设备出厂前检验': '第四节设备出厂前检验\n1.在合同设备及其相关设备的生产制造期间如需到国外进行质量检查，招标人可派出一行7\n个成员包括1名翻译到中标人国逗留14天，招标人质量检查人员的费用包括返程机票费、\n膳食和住宿费由招标人自行承担，在中标人国当地交通和安排，比如所需的安全用具、有\n关公务的技术文件和图纸、统计资料、制造和检验标准及检验中所需的其它数据由中标人\n免费提供。\n2.在制造期间招标人的一切监理和质检活动所形成的书面资料均不作为中标人产品质量证明\n文件。在交货前招标人的质检，既不能免去合同中属于投标人质量担保期范围内的责任，\n也不能替代设备抵运招标人现场的质量检验。\n3．在中检中质检团成员发现或提出的问题，双方应积极通过友好的态度协商解决。\n4.设备在出厂前供货方必须进行整体联合试运转，根据试运转时间确定招标人中检时间，联\n合试运转应在招标人中检人员监督下进行。\n5.在设备到达招标人现场后组装试运转中如出现问题，原因是中标人没有在出厂前进行设备\n整体联合试运转，因此推迟的时间将按照推迟交货期来计算。\n57',
 '标准': '第九节标准\n43',
 '安装、检验、调试、试运行及验收': '第六节安装、检验、调试、试运行及验收\n1.在该附录中：\n安装：意为招标人安装人员在中标人的服务工程人员的监督与指导下，将整套设备\n或一个系统安装起来。\n试运转：即为在空载条件下测试该设备。\n性能调试：即在它们的额定负载下测试设备，检查其是否能达到合同规定的所有技\n术性能。\n试运行：即为设备按照合同要求性能投入运转。\n验收：即为该设备达到合同规定的试运转、性能调试和试运行技术要求后招标人正\n式接收。\n2.设备到货应随机提供出厂验收报告。\n3．在设备经过试运转、性能调试、试运行之后，买卖双方对设备性能进行鉴定，符合合同\n要求，招标人出据验收证明并由中标人确认。验收标准为合同规定的要求和相关标准、\n中国国家标准、规范以及国际标准和双方认可的标准。\n129',
 '备件和工具': '第二节备件和工具\n1.所有为设备的组装、空载试验、带载试验、试运行、质保期内1年必备的备件、消耗品，\n包括专用工具、仪器、仪表等，在设备交货时提供。推迟的交货期将按照设备推迟交货\n计算。\n2.中标人应提供完整备件手册、备件件号、

In [44]:
import json
template_dict = {
    'type':'采煤机',
    'content':template
}
for extract in set_section:
    with open("../source/resultTemps/template.json","w",encoding="utf-8") as f:
        json.dump(template_dict,f,ensure_ascii=False)

In [45]:
len(template)

9

In [48]:
import json
template = json.load(open('../source/resultTemps/template.json',"r"))
template['content']

{'设备出厂前检验': '第四节设备出厂前检验\n1.在合同设备及其相关设备的生产制造期间如需到国外进行质量检查，招标人可派出一行7\n个成员包括1名翻译到中标人国逗留14天，招标人质量检查人员的费用包括返程机票费、\n膳食和住宿费由招标人自行承担，在中标人国当地交通和安排，比如所需的安全用具、有\n关公务的技术文件和图纸、统计资料、制造和检验标准及检验中所需的其它数据由中标人\n免费提供。\n2.在制造期间招标人的一切监理和质检活动所形成的书面资料均不作为中标人产品质量证明\n文件。在交货前招标人的质检，既不能免去合同中属于投标人质量担保期范围内的责任，\n也不能替代设备抵运招标人现场的质量检验。\n3．在中检中质检团成员发现或提出的问题，双方应积极通过友好的态度协商解决。\n4.设备在出厂前供货方必须进行整体联合试运转，根据试运转时间确定招标人中检时间，联\n合试运转应在招标人中检人员监督下进行。\n5.在设备到达招标人现场后组装试运转中如出现问题，原因是中标人没有在出厂前进行设备\n整体联合试运转，因此推迟的时间将按照推迟交货期来计算。\n57',
 '标准': '第九节标准\n43',
 '安装、检验、调试、试运行及验收': '第六节安装、检验、调试、试运行及验收\n1.在该附录中：\n安装：意为招标人安装人员在中标人的服务工程人员的监督与指导下，将整套设备\n或一个系统安装起来。\n试运转：即为在空载条件下测试该设备。\n性能调试：即在它们的额定负载下测试设备，检查其是否能达到合同规定的所有技\n术性能。\n试运行：即为设备按照合同要求性能投入运转。\n验收：即为该设备达到合同规定的试运转、性能调试和试运行技术要求后招标人正\n式接收。\n2.设备到货应随机提供出厂验收报告。\n3．在设备经过试运转、性能调试、试运行之后，买卖双方对设备性能进行鉴定，符合合同\n要求，招标人出据验收证明并由中标人确认。验收标准为合同规定的要求和相关标准、\n中国国家标准、规范以及国际标准和双方认可的标准。\n129',
 '备件和工具': '第二节备件和工具\n1.所有为设备的组装、空载试验、带载试验、试运行、质保期内1年必备的备件、消耗品，\n包括专用工具、仪器、仪表等，在设备交货时提供。推迟的交货期将按照设备推迟交货\n计算。\n2.中标人应提供完整备件手册、备件件号、

In [11]:
result['备件和工具']

[['第二节备件和工具',
  '1.所有为设备的组装、空载试验、带载试验、试运行、质保期内1年必备的备件、消耗品，',
  '包括专用工具、仪器、仪表等，在设备交货时提供。推迟的交货期将按照设备推迟交货',
  '计算。',
  '2.中标人应提供完整备件手册、备件件号、数量、规格型号、价格表的CD盘，随同设备发',
  '货。',
  '3.中标人应保证所有零部件均有唯一编码，如属外购标准件，要求必须按照原厂家编码执',
  '行。',
  '4.中标人还将进一步提供可靠信息以及机械与电气设备上的所需的备件、易耗品及标准件',
  '的货源地，包括润滑油脂。',
  '5.设备采用的外购、外协件应提供原产地证明及检验合格证书。',
  '6.如因为中标人提供1年期备件（不超过主机价格的5%）明细不准确，导致招标人误采购',
  '或按明细提供数量不足以满足生产需求，中标人应免费提供相应的备件。',
  '7.中标人应保证长期以最优惠的价格供给易损件和备件。如果备件发生设计变更，应将变',
  '更信息及时通知用户。',
  '8.中标人备件价格在设备开始使用的3年内必须维持稳定。',
  '9.在5年内，因中标人技术升级导致部分备件不能提供时，中标人要免费为用户升级设备。',
  '10.5年后在备件停止生产的情况下，中标人应事先将要停止生产的计划通知招标人使招标',
  '人有足够的时间采购所需的备件。',
  '11.5年后在备件停止生产后，如果招标人要求，中标人应免费向招标人提供备件的蓝图、',
  '图纸和规格。',
  '125'],
 ['第三节备件和工具',
  '1.所有为设备的组装、空载试验、带载试验、试运行、质保期内必备的备件、消耗品，包括',
  '专用工具、仪器、仪表等，在设备交货时提供。推迟的交货期将按照设备推迟交货计算。',
  '2.中标人应提供完整备件手册、备件件号、数量、规格型号、价格表的CD盘，随同设备发货。',
  '3.中标人应保证所有零部件均有唯一编码，如属外购标准件，要求必须按照原厂家编码执行。',
  '4.中标人还将进一步提供可靠信息以及机械与电气设备上的所需的备件、易耗品及标准件的',
  '货源地，包括润滑油脂。',
  '5.设备采用的外购、外协件应提供原产地证明及检验合格证书。',
  '6.如因为中标人提

In [12]:
result['供货范围、技术规格、参数与要求']

[['第一节供货范围、技术规格、参数与要求',
  '第一包第1-1项：采煤机（截割功率650kW）',
  '第一部分货物需求一览表',
  '单数交货',
  '序号名称规格型号备注',
  '位量期',
  '2018',
  '采煤机1台年8月梅花井煤矿',
  '30日',
  '650*2+125*2+20*2',
  '≥MG650/',
  '1采煤机+160（进口电机≥',
  '1750-WD',
  '620/1660）',
  '左右摇臂、左右牵引装置配',
  '套电气设备、各个驱动装置齿轨轮节距',
  '1.1套1',
  '的驱动电机、液压系统、截176mm',
  '割滚筒（包括截齿）等',
  '顺槽数据传输、接收及显示含缆线及连接件和',
  '1.2套1',
  '装置数据传输软件',
  '自动截割、惯性导航控制及',
  '1..3套1',
  '软件',
  '1.4摄像系统套1',
  '保证150mm2进',
  '线，200米，选用',
  '1.5电缆进线拖曳装置O2C套1霸州华宇或神华泉',
  '州公司或同等质量',
  '产品',
  '选用虎牌电缆或同',
  '等品质进口电缆，',
  '100',
  '1.6电缆150mm2米芯线含高强度通信',
  '0',
  '光纤、带电缆连接',
  '器',
  'DN50/38MP',
  '1.7水管米400',
  'a',
  '2随机大部件',
  '摇臂(左右摇臂各一件，含',
  '2.1对1备用',
  '连接件)',
  '牵引部及行走箱（左右各一',
  '2.2对1备用',
  '套）',
  '2.3变频器套1备用',
  '2.4调高油缸件2备用',
  '44',
  '2.5调高泵台1备用',
  '2.6截割滚筒（左右含截齿）Φ1800mm对1备用',
  '2.7牵引电动机台1备用',
  '2.8截割电动机台1备用',
  '2.9油泵电动机台1备用',
  '3备品备件（序号1总价5%）批1',
  '4专用工具套2高压泵一台',
  '5随机技术资料套66套',
  '第二部分工作环境',
  '（一）工作面煤层情况',
  '梅花井煤矿1110208工作面回采走向长4500m，倾斜长约为217m；煤层厚度为3～4m

In [13]:
result.keys()

dict_keys(['设备出厂前检验', '标准', '安装、检验、调试、试运行及验收', '备件和工具', '技术资料和图纸', '设计联络会及配套责任', '技术服务', '供货范围、技术规格、参数与要求', '质量保证'])

In [20]:
origin = pdfplumber.open("../source/datas/1.pdf")
origin.metadata

{'Producer': 'PyPDF2'}

In [21]:
import PyPDF2

In [37]:
origin = PyPDF2.PdfFileReader("../source/datas/xx.pdf")
page = origin.getPage(3)
page.artBox

RectangleObject([0, 0, 595.5, 842.25])

In [38]:
page.bleedBox

RectangleObject([0, 0, 595.5, 842.25])

In [39]:
page.extractText()

' \n \n \n \n1.9\n\n \n\nCA\n\n \n \n \n \n \n2.5\n\n \n\n \n \n \n \n \n2.9\n\n \n\n \n \n \n \n \n3.1\n\n \n\n\n \n \n \n \n \n3.2\n\n \n\n \n6.\n˘Å˚7˙·˚7\n \n \n \n \n \n6.1 \n˘Å˘Z!™˝&˝&L$˘Z!™˝&\n\n2019\n-\n08\n-\n13 09:30\n˝&L$˘Z!™˝&\n5†˘Å˚7\n\n \n \n \n \n \n6.2 \n˚75†˘Å\n\n \n \n \n \n \n6.3\n˚75†˘Å˚7\n\n \n7.\n\n \n \n \n \n \n\n5†\n,\n\n\n \n \n8.\n\n \n \n\n \n9.\n\n \n \n\n \n\n \n\n \n˛~;¤TU\n \n\n \n0473\n-\n5678735\n \n \n\n \n\n \n\n \n \n\n \n\n3\n\n6\n\n \nFÞ\n \n \n\n \n100007\n \n\n \n\n \n+e\n \n \n\n \n010\n-\n58134664\n \n\n \n \n\n \n010\n-\n58132371\n \n\n \n13707509@shenhua.cc\n \n'

In [141]:
def match_para(origin):
    paras = {}
    pattern = re.compile(r'(技术｜要求|参数｜规格)')
    for section,data in origin['content'].items():
        if re.match(r'.*(技术|要求|参数|规格).*',section):
            reg1 =r'.*(:|：).*(。|\s)'
            signal=':[BLANK]'
            groups = re.finditer(reg1,data)
            for group in groups:
                start,end = group.span()
                para = data[start:end+1]
                if ":" in para:
                    temp = para.split(":")
                else:
                    temp = para.split("：")
                paras[temp[0]] = temp[1]
    return paras
#             new_text1 = re.sub(reg1, signal, data)
            

In [142]:
match_para(template)

{'6.中标人要提供下列相关的技术资料及图纸': '\n总',
 '任与任务如下': '\n—',
 '3.中标人将提供所有的关于装配与组装所用的专用工具,例如': '专用测试仪、测量仪和机械工\n具',
 '一、工作面情况': '\n1',
 '二、水文地质情况': '\nＩ',
 '△1.1生产能力': '≥4000t/h。\n1',
 '1.2适应煤的单向抗压强度': '≥40MPa。\n1',
 '1.3适应夹矸的单向抗压强度': '50MPa～100MPa。\n△',
 '△1.4适应的工作面倾角': '0～25°。\n△',
 '△1.5适应的工作面走向倾角': '±0～17°（根据现场实际调整导向滑靴尺寸和支撑滑\n靴',
 '△1.6采高': '3.2-5.9m。\n△',
 '△1.7采煤机过煤间隙': '≥1200mm。\n△',
 '△1.8卧底量': '≥650mm。\n△',
 '△1.9供电电源': '3300V(±12%)，50Hz。\n1',
 '△1.11装机总功率': '≥2490kW。\n*',
 '*1.12截割功率': '≥2×900kW。\n*',
 '*1.13交流牵引电机功率': '≥2×200kW。\n△',
 '△1.14破碎机功率': '≥250kW。\n△',
 '△1.15泵电机总功率': '≥40kW。\n△',
 '△1.17摇臂大修周期': '≥6Mt过煤量。\n△',
 '△1.18破碎机大修周期': '≥5Mt过煤量。\n△',
 '△1.19破碎机寿命': '≥18Mt过煤量。\n△',
 '△1.20牵引块大修周期': '≥5Mt过煤量（含链轮）。\n△',
 '△1.21要求整机质保期': '≥5Mt过煤量。\n△',
 '△1.22整机大修周期': '≥6Mt过煤量(整机大修周期的范围包括主机架、电气系统、\n牵',
 '△1.23整机寿命': '≥25Mt过煤量。\n△',
 '△1.24整机重量': '约135t。\n*',
 '*1.25采煤机机身高度': '≤2350mm（不含机身顶护板，矮机身≤1850mm）\n8',
 '*1.27最大牵引速度': '煤质硬度为f=2～3时，重载时不小于12m/min，空载时不小\n于',
 '2.技术要求': '\n*',
 '电控箱及自动化应

In [93]:
re.match(r'.*(技术|要求|参数|规格).*',"啊 d 技术")

<re.Match object; span=(0, 6), match='啊 d 技术'>